In [1]:
import numpy as np
import collections
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animate
from IPython.display import HTML
%matplotlib inline

### Create Brownian bug function

In [2]:
def brownian_bug(N, birth, death, step, A, phi, theta, df):
    bug_no = np.array(df['bug_no'].tolist(), float)
    pos_x = np.array(df['pos_x'].tolist(), float)
    pos_y = np.array(df['pos_y'].tolist(), float)
    
    counter = len(bug_no)
    temp_death = 0
    temp_birth = 0

    for i in range(counter):
        chance = np.random.random()
        if chance <= birth:
            bug_no = np.append(bug_no, max(bug_no) + 1)
            pos_x = np.append(pos_x, pos_x[i-temp_death])
            pos_y = np.append(pos_y, pos_y[i-temp_death])
            temp_birth += 1
        elif chance > birth and chance <= birth+death:
            bug_no = np.delete(bug_no, i-temp_death)
            pos_x = np.delete(pos_x, i-temp_death)
            pos_y = np.delete(pos_y, i-temp_death)
            temp_death += 1

    if len(bug_no) == 0:
        return print("Network: extinct")

    for k in range(len(pos_x)):
        angle = np.random.randint(0,361)
        angle = angle*np.pi/180

        pos_x[k] += step*np.cos(angle)
        pos_y[k] += step*np.sin(angle)

        #advection Stanford map
        pos_x[k] = pos_x[k] + ((A/(2*np.pi)) * np.sin(2*np.pi*pos_y[k]))
        pos_y[k] = pos_y[k] + pos_x[k]

        while pos_x[k] < 0 or pos_x[k] > 1 or pos_y[k] > 1 or pos_y[k] < 0:
            if pos_x[k] < 0: pos_x[k] +=1
            if pos_x[k] > 1: pos_x[k] -=1
            if pos_y[k] < 0: pos_y[k] +=1
            if pos_y[k] > 1: pos_y[k] -=1
        
    
    d = {'bug_no': bug_no, 'pos_x': pos_x, 'pos_y': pos_y}
    dataframe = pd.DataFrame(d)
    
    return dataframe

### Create network dataframe

In [3]:
def distance(po_0,po_1, p1_0, p1_1):
    return np.sqrt((po_0 - p1_0)**2 + (po_1 - p1_1)**2)

def create_network_dataframe(dataframe):
    df = dataframe
    distances = 0
    transmission_radius = 0.1
    Network = []
    for i in range(len(df)):
        #if i%200==0: print('running')
        for j in range(len(df)):
            if i == j:
                continue
            else:
                distances = distance(df['pos_x'][i],df['pos_y'][i],df['pos_x'][j],df['pos_y'][j])
                if distances <= transmission_radius: 
                    edge = 1
                    dist = distances
                    cost = distances**2
                else: 
                    edge = 0
                    dist = distances
                    cost = 0

                Network.append({'1_node': i, '1_node_val_x': df['pos_x'][i], '1_node_val_y': df['pos_y'][i], '2_node': j, 
                                '2_node_val_x': df['pos_x'][j], '2_node_val_y': df['pos_y'][j], 'edge': edge, 'cost': cost,
                                'distance': dist})

    df = pd.DataFrame(Network)
    return df

### Create network

In [41]:
def create_network(dataframe):
    network = dataframe
    G_original_network = nx.Graph()

    n1 = network['1_node'].tolist()
    n2 = network['2_node'].tolist()
    nvalsx = network['1_node_val_x'].tolist()
    nvalsy = network['1_node_val_y'].tolist()
    nvalsx2 = network['2_node_val_x'].tolist()
    nvalsy2 = network['2_node_val_y'].tolist()
    edge = network['edge'].tolist()

    for i in range(len(n1)):
        G_original_network.add_node(int(n1[i]), pos = (nvalsx[i], nvalsy[i]))

    for i in range(len(n2)):
        G_original_network.add_node(int(n2[i]), pos = (nvalsx2[i], nvalsy2[i]))

    pos=nx.get_node_attributes(G_original_network,'pos')

    for i in range(len(n1)):
        if edge[i] != 0:
            G_original_network.add_edge(int(n1[i]),int(n2[i]),weight=float(edge[i]))
    
    return G_original_network, pos

### Visualize network

In [42]:
def plot_network(G_original_network, pos):
    plt.figure(figsize=(3, 3))
    nx.draw_networkx(G_original_network, pos,with_labels=False,edge_color='blue',style = 'solid',width = 0.7, node_size = 8, node_color='blue')
    plt.draw()